## Homework 12

#### *DATA 1010*

In [1]:
import Pkg; Pkg.activate(".")

Activating environment at `~/Google Drive/DSI/Masters Program/DATA 1010/DATA1010-2019/problem-sets/hw12/Project.toml`


In [2]:
using Random, CSV, Turing, Distributions, MCMCChains, Plots, StatsPlots, StatsFuns
Turing.turnprogress(false);
Random.seed!(123);

┌ Info: [Turing]: global PROGRESS is set as false
└ @ Turing /Users/sswatson/.julia/packages/Turing/xFzfF/src/Turing.jl:23


---

## Problem 1

Consider two probability density functions, $f_1(x)$ and $f_2(x)$. Assume that we have $N$ observations $x_1,\cdots,x_N$ drawn from a mixture of these **known** densities:
$$ \pi f_1(x) + (1-\pi) f_2(x) $$
Write down the E and M steps you would use to estimate the proportion $\pi$, given the dataset.

*Hint*: Use a hidden state variable $z_i$ to indicate that data point $x_i$ belongs to $f_1$.

---



## Problem 2 (Bayesian Logistic Regression)

Recall logistic regression, where we find coefficients $\mathbf{\beta}, \alpha$ to predict label the $y_i$ given input features $\mathbf{x_i}$:
$$ \mathbb{P}(y_i = 1|\mathbf{x_i}) = \sigma(\boldsymbol{\beta}'\mathbf{x_i} + \alpha)$$

Here, we develop the Bayesian counterpart to obtain the full posterior over the coefficients: $\mathbb{P}(\mathbf{\beta}|\mathcal{D})$, where data $\mathcal{D}$ consists of all training observations. Unlike linear regression, there is no convenient conjugate prior for logistic regression, so we use MCMC with `Turing.jl`

In [3]:
grad_data = CSV.read("grad_admission.csv")
first(grad_data, 6)

,admit,gre,gpa,rank
,Int64,Int64,Float64,Int64
1,0,380,3.61,3
2,1,660,3.67,3
3,1,800,4.0,1
4,1,640,3.19,4
5,0,520,2.93,4
6,1,760,3.0,2


In [4]:
grad_data.gre = (grad_data.gre .- mean(grad_data.gre)) ./ std(grad_data.gre)
grad_data.gpa = (grad_data.gpa .- mean(grad_data.gpa)) ./ std(grad_data.gpa)

grad_data = grad_data[shuffle(1:size(grad_data, 1)),:];

train_test_split_index = Int(round(size(grad_data, 1) * 0.7))
train = grad_data[1:train_test_split_index, :]
test = grad_data[(train_test_split_index+1):end, :]

train_label = train[:,:admit];
test_label = test[:,:admit];

train = train[:,[:gre, :gpa, :rank]];
test = test[:,[:gre, :gpa, :rank]];

Write a Turing.jl model called `logistic_regression` which gives priors of $N(0, \sigma)$ to `intercept`, `gre`, `gpa`, and `rank`. 

In [ ]:
@model logistic_regression(x, y, n, σ) = begin
    

end;

In [5]:
# Retrieve the number of observations.
n = size(train, 1)

# Sample using HMC
chain = mapreduce(c -> sample(logistic_regression(train, train_label, n, 1), HMC(0.05, 10), 1500),
    chainscat,
    1:3
)

describe(chain)

2-element Array{ChainDataFrame,1}

Summary Statistics
. Omitted printing of 1 columns
│ Row │ parameters │ mean     │ std       │ naive_se    │ mcse       │ ess     │
│     │ Symbol     │ Float64  │ Float64   │ Float64     │ Float64    │ Any     │
├─────┼────────────┼──────────┼───────────┼─────────────┼────────────┼─────────┤
│ 1   │ gpa        │ 0.321881 │ 0.356509  │ 0.00531453  │ 0.0534671  │ 18.0723 │
│ 2   │ gre        │ 0.59765  │ 0.0950642 │ 0.00141713  │ 0.0111314  │ 18.0723 │
│ 3   │ intercept  │ 2.65669  │ 0.130045  │ 0.0019386   │ 0.0169787  │ 18.0723 │
│ 4   │ rank       │ 0.269729 │ 0.0652298 │ 0.000972388 │ 0.00895466 │ 18.0723 │

Quantiles

│ Row │ parameters │ 2.5%      │ 25.0%     │ 50.0%    │ 75.0%    │ 97.5%    │
│     │ Symbol     │ Float64   │ Float64   │ Float64  │ Float64  │ Float64  │
├─────┼────────────┼───────────┼───────────┼──────────┼──────────┼──────────┤
│ 1   │ gpa        │ -0.160595 │ -0.160595 │ 0.445086 │ 0.688054 │ 0.688054 │
│ 2   │ gre        │ 0.

---

## Problem 3 (Simpson’s Paradox)

Consider the following classic example given by Edward Simpson in 1951.

700 patients were given access to a drug the effectiveness of which we would like to study. A total of 350 patients chose to take the drug and 350 didn’t. The result is given below:

|Categories            | Drug                           | No Drug   |
|---------------    |---------------------|--------------------------------|
| Men           | 81 out of 87 (93%)             | 234 out of 270 recovered (87%) |   
| Women         | 192 out of 263 recovered (73%) | 55 out of 80 recovered (69%)   |
| Combined data | 273 out of 350 recovered (78%) | 289 out of 350 recovered (83%) |

(a) Not controlling for sex, describe the overall trend of taking the drug.

(b) Describe the overall trend of taking the drug conditioned on sex.

(c) Match the following English sentences to its corresponding mathematical statement in the list below. Note that one of the mathematical statements will not have a match. Here men are $Z=0$, women are $Z=1$, non-recovery is $Y=0$, and recovery is $Y=1$, and non-treatment is $X=0$ and treatment is $X=1$.

(i) The drug is beneficial for men.  
(ii) The drug is beneficial for women.  
(iii) The drug is beneficial overall.

(I) $\mathbb{P}(Y=1 | X=1) > \mathbb{P}(Y=1 | X=0)$  
(II) $\mathbb{P}(C_1 = 1) > \mathbb{P}(C_1 = 0)$  
(III) $\mathbb{P}(C_1 = 1 | Z = 1) > \mathbb{P}(C_1 = 0 | Z = 1)$  
(IV) $\mathbb{P}(C_1 = 1 | Z = 0) > \mathbb{P}(C_1 = 0 | Z = 0)$

(d) Show mathematically that if a drug is beneficial for each subpopulation in a partition of the overall population, then it is beneficial overall. (Hint: you will want to use the formulation of "beneficial" from part (c). 

---

## Problem 4

Consider the following statement from an advertisement. 

‘In this town we observe that those who take soy isoflavone supplements have improved cognitive performance. Thus, for anyone looking to improve their cognitive performance, they should consider taking soy isoflavone.’

Identify the problematic causal inference in this statement. Identify whether each statement strengthens or weakens the claim (or neither).


a) The alleged cause and effect are both effects of some common cause.

b) The cause and effect are flipped; the alleged cause is the effect and vice versa.

c) The presence of the cause in a relatively large population does not coincide with the detection of the effect. 

d) The presence of the effect has been identified in a large population but not the cause.

e) A possible confounder for this effect has been ruled out.

---

## Problem 5

Suppose we want to evaluate the impact of the National Supported Work (NSW) Demonstration, a labor training program on real earnings in 1978. Our dataset, from [LaLonde (1986)](https://www.jstor.org/stable/1806062), consists of measurements of age, years of schooling, ethnicity, marital status and earnings.

In [6]:
la_londe = CSV.read("lalonde.csv")
la_londe = la_londe[shuffle(1:size(la_londe, 1)),:];
first(la_londe, 6)

,data_id,treat,age,education,black,hispanic,married,nodegree,re75
,String,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64
1,Lalonde Sample,0,18,9,0,1,0,1,2684.91
2,Lalonde Sample,0,33,11,1,0,0,1,0.0
3,Lalonde Sample,0,19,9,1,0,0,1,1537.93
4,Lalonde Sample,0,20,9,1,0,0,1,0.0
5,Lalonde Sample,1,46,8,1,0,1,1,0.0
6,Lalonde Sample,1,31,13,0,0,0,0,5498.35


a) Compare features (age, years of education, etc.) between individuals that received treatment (`treat = 1`) and those who are in the control group (`treat = 0`). What do you observe? What might be the confounding factors in our analysis?

To mitigate (but not entirely eliminate) the effect of confounding factors, we can match data between treatment and control pairs as follows:

i. Define a distance measure between feature vectors.

ii. Select a subset from the treatment and control groups which are close.

iii. Evaluate quality of matching

iv. Perform analysis on the matched set.

b) Consider the *Mahalanobis distance* between observations $x_i$ and $x_j$, defined as : $D_{ij} = (x_i - x_j)^T\Sigma^{-1}(x_i - x_j)$, where $\Sigma$ is the covariance matrix estimated from the data. Compute $D_{ij}$ using age, education and re75 (real income in 1975). 

c) The *propensity score distance* is defined as $D_{ij} = |\pi_i - \pi_j|$, where $\pi_i = \mathbb{P}(\text{treat}=1|x_i)$ is the probability of being in the treatment group for observation $x_i$. Compute propensity scores for the LaLonde dataset and extract a matching set.